In [6]:
%pip install redis

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [21]:
import redis
import logging
from langchain.vectorstores.redis import Redis
from langchain.embeddings import OpenAIEmbeddings

In [20]:
#embeddings = OpenAIEmbeddings()

#connection = redis.Redis()
#connection.ping()
r = redis.Redis(host='localhost', port=6379, decode_responses=True, db=0)

#class Redis(object):#
#    def __init__(self, host='localhost', port=6379,db=0, password=None, socket_timeout=None)

In [ ]:
#Test connection
try:
    r.set('foo', 'bar')
    value = r.get('foo')
    print(value)
except redis.ConnectionError as e:
    print(f"Error connecting to Redis: {e}")


#run redis-server in terminal
#redis-cli for host:port pair -> test connectivity with "PING"
#to close: redis-cli shutdown

#MSET to set multiple key:value pairs and MGET to get multple values for keys

In [16]:
r.mset({"Croatia": "Zagreb", "Bahamas": "Nassau"})

r.get("Bahamas")



'Nassau'

In [8]:
metadata = [
    {
        "user": "john",
        "age": 18,
        "job": "engineer",
        "credit_score": "high",
    },
    {
        "user": "derrick",
        "age": 45,
        "job": "doctor",
        "credit_score": "low",
    },
    {
        "user": "nancy",
        "age": 94,
        "job": "doctor",
        "credit_score": "high",
    },
    {
        "user": "tyler",
        "age": 100,
        "job": "engineer",
        "credit_score": "high",
    },
    {
        "user": "joe",
        "age": 35,
        "job": "dentist",
        "credit_score": "medium",
    },
]

texts = ["foo", "foo", "foo", "bar", "bar"]

In [22]:
logging.basicConfig()

In [ ]:

rds = Redis.from_texts(
    texts,
    embeddings, 
    metadatas=metadata,
    redis_url="redis://localhost:6379",
    index_name="users",
)

rds.index_name

